# Dependencies

In [1]:
%%capture
!pip install pandas
!pip install pyarrow
!pip install tensorflow
!pip install protobuf==3.20.*
!pip install mediapipe==0.9.0.1

In [2]:
import os
import json
import shutil
import random
import matplotlib
import numpy as np
from itertools import chain
from collections import deque
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from collections import Counter
from matplotlib import animation, rc
from IPython.display import display, Image

import cv2
import pandas as pd
import mediapipe as mp
import tensorflow as tf
from tensorflow import keras
import pyarrow.parquet as pq
from tensorflow.keras import layers
from mediapipe.framework.formats import landmark_pb2

In [3]:
!python --version
print("TensorFlow v" + tf.__version__)
print("Mediapipe v" + mp.__version__)

Python 3.9.13
TensorFlow v2.14.0
Mediapipe v0.9.0.1


In [4]:
random.seed(42)

# Load data

In [5]:
dataset_df = pd.read_csv('kaggle_dataset/train.csv')
print("Full train dataset shape is {}".format(dataset_df.shape))

Full train dataset shape is (94477, 4)


In [6]:
dataset_df.head()

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


In [7]:
def random_sequence():
    # Fetch sequence_id, file_id, phrase from first row
    path, sequence_id, sign = dataset_df.sample(n=1).iloc[0][['path', 'sequence_id', 'sign']]
    print(f"sequence_id: {sequence_id}, path: {path}, sign: {sign}")
    
    # Fetch data from parquet file (This dataset stores each landmark in a separate row)
    sample_sequence_df_wrong_format = pq.read_table(f"kaggle_dataset/{path}",).to_pandas()
    print("Full sequence dataset shape is {}".format(sample_sequence_df_wrong_format.shape))
    
    return sample_sequence_df_wrong_format

In [8]:
random_sequence()

sequence_id: 1720683369, path: train_landmark_files/32319/1720683369.parquet, sign: toy
Full sequence dataset shape is (11403, 7)


,frame,row_id,type,landmark_index,x,y,z
0,27,27-face-0,face,0,0.423072,0.284930,-0.046184
1,27,27-face-1,face,1,0.400122,0.235108,-0.074740
2,27,27-face-2,face,2,0.406785,0.248177,-0.041294
3,27,27-face-3,face,3,0.381954,0.198252,-0.050777
4,27,27-face-4,face,4,0.396893,0.223221,-0.078449
...,...,...,...,...,...,...,...
11398,47,47-right_hand-16,right_hand,16,NaN,NaN,NaN
11399,47,47-right_hand-17,right_hand,17,NaN,NaN,NaN
11400,47,47-right_hand-18,right_hand,18,NaN,NaN,NaN
11401,47,47-right_hand-19,right_hand,19,NaN,NaN,NaN


In [9]:
# Check if all frames contain all coords
# for i, row in tqdm(dataset_df.iterrows()):
#     path, sequence_id, sign = row[['path', 'sequence_id', 'sign']].values
#     l = len(pq.read_table(f"kaggle_dataset/{path}",).to_pandas())
#     if l % 543 != 0:
#         print(l/543)

In [10]:
NUM_COORDS_FRAME = 543

def gen_column_names(prefix: str, name: str, count: int):
    col_names = []
    for i in range(0, count+1):
        col_names.append(f"{prefix}_{name}_{i}")
    return col_names

columns = list(chain(
    gen_column_names("x", "face", 467),
    gen_column_names("x", "left_hand", 20),
    gen_column_names("x", "pose", 32),
    gen_column_names("x", "right_hand", 20),
    gen_column_names("y", "face", 467),
    gen_column_names("y", "left_hand", 20),
    gen_column_names("y", "pose", 32),
    gen_column_names("y", "right_hand", 20),
    gen_column_names("z", "face", 467),
    gen_column_names("z", "left_hand", 20),
    gen_column_names("z", "pose", 32),
    gen_column_names("z", "right_hand", 20),
))

# Transform the dataframe to be tha same format as the fingerspelling dataset (1 row for each frame)
def transform_df(df):
    x = df.x
    y = df.y
    z = df.z

    num_frames = len(df) // NUM_COORDS_FRAME
    x = x.values.reshape(num_frames, NUM_COORDS_FRAME)
    y = y.values.reshape(num_frames, NUM_COORDS_FRAME)
    z = z.values.reshape(num_frames, NUM_COORDS_FRAME)

    data = np.concatenate((x, y, z), axis=1)
        
    return pd.DataFrame(data, columns=columns)

In [11]:
%%time
transform_df(random_sequence()).head()

sequence_id: 1580474962, path: train_landmark_files/37055/1580474962.parquet, sign: police
Full sequence dataset shape is (10317, 7)
CPU times: total: 0 ns
Wall time: 90.1 ms


,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,x_face_9,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
0,0.440979,0.423166,0.435760,0.420858,0.421695,0.425599,0.439508,0.375274,0.442172,0.440776,...,-0.114254,-0.137291,-0.095169,-0.127179,-0.149179,-0.166778,-0.140523,-0.171021,-0.188343,-0.202747
1,0.435873,0.417833,0.430695,0.415879,0.416348,0.420413,0.434845,0.373704,0.437454,0.435925,...,-0.034662,-0.048832,-0.033992,-0.048368,-0.058628,-0.067475,-0.069385,-0.080855,-0.084877,-0.088867
2,0.434042,0.415535,0.429008,0.414493,0.414078,0.418449,0.433815,0.372085,0.436690,0.435121,...,0.012645,0.001074,-0.022365,-0.028802,-0.035450,-0.043039,-0.074882,-0.082073,-0.084526,-0.088733
3,0.440385,0.421237,0.434195,0.419117,0.419683,0.423618,0.437731,0.374317,0.440360,0.438791,...,-0.018938,-0.030653,-0.030443,-0.042757,-0.050660,-0.057957,-0.069682,-0.081162,-0.085395,-0.089679
4,0.443920,0.422665,0.435808,0.420696,0.421123,0.425137,0.439400,0.376783,0.442003,0.440365,...,-0.035398,-0.049149,-0.032021,-0.050425,-0.063441,-0.073261,-0.063813,-0.081788,-0.090724,-0.097689


# Animation preview

In [12]:
matplotlib.rcParams['animation.embed_limit'] = 2**128
matplotlib.rcParams['savefig.pad_inches'] = 0
rc('animation', html='jshtml')

def create_animation(images):
    fig = plt.figure(figsize=(6, 9))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    im=ax.imshow(images[0], cmap="gray")
    plt.close(fig)
    
    def animate_func(i):
        im.set_array(images[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames=len(images), interval=1000/10)

mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles

def collect_landmarks_oftype(_type: str, frame_data: pd.Series):
    x_type = frame_data.filter(regex=f"x_{_type}.*").values
    y_type = frame_data.filter(regex=f"y_{_type}.*").values
    z_type = frame_data.filter(regex=f"z_{_type}.*").values
    landmarks = landmark_pb2.NormalizedLandmarkList()
    for x, y, z in zip(x_type, y_type, z_type):
        landmarks.landmark.add(x=x, y=y, z=z)
    return landmarks
        
def produce_video_from_seq(seq_df: pd.DataFrame):
    frames = []
    for seq_idx in range(len(seq_df)):
        frame = np.zeros((600, 600, 3))
        frame_data = seq_df.iloc[seq_idx]
        
        # Right hand
        right_hand_landmarks = collect_landmarks_oftype("right_hand", frame_data)
        mp_drawing.draw_landmarks(
                frame,
                right_hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style())
        
        # Left Hand
        left_hand_landmarks = collect_landmarks_oftype("left_hand", frame_data)
        mp_drawing.draw_landmarks(
                frame,
                left_hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style())
        
        # Face
        face_landmarks = collect_landmarks_oftype("face", frame_data)
        mp_drawing.draw_landmarks(
            frame,
            face_landmarks,
            mp_holistic.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
        
        # Pose
        pose_landmarks = collect_landmarks_oftype("pose", frame_data)
        mp_drawing.draw_landmarks(
            frame,
            pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.
            get_default_pose_landmarks_style())
        
        frames.append(frame.astype(np.uint8))
    return frames

In [13]:
video = produce_video_from_seq(transform_df(random_sequence()))
create_animation(video)

sequence_id: 1832432929, path: train_landmark_files/26734/1832432929.parquet, sign: hat
Full sequence dataset shape is (17919, 7)


# Statistics

In [14]:
def lengths_hist(lengths: list, title="", space=50):
    plt.hist(lengths, np.linspace(0, space, space))
    plt.ylim(plt.ylim())
    max_length = max(lengths)
    plt.plot([max_length, max_length], plt.ylim())
    plt.title(f'{title} (Max: {max_length})')
    plt.show()

def calc_percentiles(values: list, percentiles=[25, 50,75, 80, 90]):
    percentile_values = np.percentile(values, percentiles)
    for p, value in zip(percentiles, percentile_values):
        print(f"{p}th percentile: {value}")

In [ ]:
seq_lengths = []

for file_path in tqdm(dataset_df.sample(n=3000).path):
    count = pq.read_table("kaggle_dataset/" + file_path, columns=['frame']).num_rows
    seq_lengths.append(count // NUM_COORDS_FRAME)

lengths_hist(seq_lengths, "Length of sequences", 100)
calc_percentiles(seq_lengths)

# Save to TFRecords

In [15]:
# Pose coordinates for hand movement.
LPOSE = [13, 15, 17, 19, 21]
RPOSE = [14, 16, 18, 20, 22]
FPOSE = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
POSE = LPOSE + RPOSE + FPOSE

X = [f'x_right_hand_{i}' for i in range(21)] + [f'x_left_hand_{i}' for i in range(21)] + [f'x_pose_{i}' for i in POSE]
Y = [f'y_right_hand_{i}' for i in range(21)] + [f'y_left_hand_{i}' for i in range(21)] + [f'y_pose_{i}' for i in POSE]
Z = [f'z_right_hand_{i}' for i in range(21)] + [f'z_left_hand_{i}' for i in range(21)] + [f'z_pose_{i}' for i in POSE]

FEATURE_COLUMNS = X + Y + Z

X_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if "x_" in col]
Y_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if "y_" in col]
Z_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if "z_" in col]

RHAND_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if "right" in col]
LHAND_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if  "left" in col]
RPOSE_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if  "pose" in col and int(col.split("_")[-1]) in RPOSE]
LPOSE_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if  "pose" in col and int(col.split("_")[-1]) in LPOSE]
FPOSE_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if  "pose" in col and int(col.split("_")[-1]) in FPOSE]

MID_POINT_IDX = [i for i, col in enumerate(FEATURE_COLUMNS)  if  "pose" in col and int(col.split("_")[-1]) == 1]

In [16]:
def save_tfrecords_to(folder: str):
    for part_id, group_df in tqdm(dataset_df.groupby(by="participant_id")):
        # Save each participant's data into separate files
        tf_file = f"{folder}/{part_id}.tfrecord"
        with tf.io.TFRecordWriter(tf_file) as file_writer:
            for file_path, sign in tqdm(zip(group_df.path, group_df.sign)):
                pq_file = pq.read_table(f"kaggle_dataset/{file_path}",columns=["x", "y", "z"])

                df_original_format = pq_file.to_pandas()
                formatted_df = transform_df(df_original_format)
                frames = formatted_df.values

                features = {formatted_df.columns[i]: tf.train.Feature(float_list=tf.train.FloatList(value=frames[:, i])) for i in range(len(formatted_df.columns))}
                features["phrase"] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(sign, 'utf-8')]))
                record_bytes = tf.train.Example(features=tf.train.Features(feature=features)).SerializeToString()
                file_writer.write(record_bytes)

In [17]:
folder_name = "train_tfrecords_all_coords"
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)
    save_tfrecords_to(folder_name)
else:
    print(f"Warning: Tfrecords already exist! Delete the {folder_name} folder to regenerate them!")

# Fetch from TFRecords

In [18]:
tf_records = dataset_df.participant_id.map(lambda x: f'{folder_name}/{x}.tfrecord').unique()
print(f"List of {len(tf_records)} TFRecord files.")

List of 21 TFRecord files.


In [19]:
def decode_fn(record_bytes):
    schema = {COL: tf.io.VarLenFeature(dtype=tf.float32) for COL in FEATURE_COLUMNS}
    schema["phrase"] = tf.io.FixedLenFeature([], dtype=tf.string)
    features = tf.io.parse_single_example(record_bytes, schema)
    phrase = features["phrase"]
    landmarks = ([tf.sparse.to_dense(features[COL]) for COL in FEATURE_COLUMNS])
    # Transpose to maintain the original shape of landmarks data.
    landmarks = tf.transpose(landmarks)
    
    return landmarks, phrase

In [20]:
sorted_signs = sorted(set(dataset_df.sign))
sign_to_num = {sign : i for i, sign in enumerate(sorted_signs)}
num_to_sign = {i : sign for sign, i in sign_to_num.items()}

In [22]:
MAX_FRAME_LEN = 128
FEATURE_LEN = len(FEATURE_COLUMNS)//3
CHANNELS = 6*FEATURE_LEN

def tf_nan_mean(x, axis=0, keepdims=False):
    return tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), x), axis=axis, keepdims=keepdims) / tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), tf.ones_like(x)), axis=axis, keepdims=keepdims)

def tf_nan_std(x, center=None, axis=0, keepdims=False):
    if center is None:
        center = tf_nan_mean(x, axis=axis,  keepdims=True)
    d = x - center
    return tf.math.sqrt(tf_nan_mean(d * d, axis=axis, keepdims=keepdims))
   
def preprocess_landmarks(inp):
    x = inp[:, 0*FEATURE_LEN: 1*FEATURE_LEN]
    y = inp[:, 1*FEATURE_LEN : 2*FEATURE_LEN]
    z = inp[:, 2*FEATURE_LEN : 3*FEATURE_LEN]
    x = tf.concat([x[..., tf.newaxis], y[..., tf.newaxis], z[..., tf.newaxis]], axis=-1)
    # x.shape == (T, P, C) or (N, T, P, C)
    # N: Batch size
    # T: Sequence length
    # P: Number of points (or landmarks)
    # C: Number of channels (features) per point
    
    if tf.rank(x) == 3:
        x = x[None,...]
    # x.shape == (N, T, P, C)
    
    # Mean of midpoints through timesteps
    mean = tf_nan_mean(tf.gather(x, [MID_POINT_IDX[0]], axis=2), axis=[1,2], keepdims=True)
    mean = tf.where(tf.math.is_nan(mean), tf.constant(0.5,x.dtype), mean)
    # x.shape == (N, 1, 1, C)
    
    # Filtering for relevant points goes here
    # I only saved the relevant ones by chance so experiment with other combinations
    
    std = tf_nan_std(x, center=mean, axis=[1,2], keepdims=True)
    # std.shape == (N, 1, 1, C)
    
    x = (x - mean)/std
    # x.shape == (N, T, P, C)
    
    if MAX_FRAME_LEN is not None:
        x = x[:,:MAX_FRAME_LEN]
    seq_length = tf.shape(x)[1]
    x = x[...,:2]
    # x.shape == (N, T, P, 2)

    # Calc differences between consequtive frames and with skipping 1 in between
    dx = tf.cond(tf.shape(x)[1]>1,lambda:tf.pad(x[:,1:] - x[:,:-1], [[0,0],[0,1],[0,0],[0,0]]),lambda:tf.zeros_like(x))
    dx2 = tf.cond(tf.shape(x)[1]>2,lambda:tf.pad(x[:,2:] - x[:,:-2], [[0,0],[0,2],[0,0],[0,0]]),lambda:tf.zeros_like(x))
    # dx.shape == dx2.shape == (N, T, P, 2)
    
    x = tf.concat([
        tf.reshape(x, (-1,seq_length,2*FEATURE_LEN)),
        tf.reshape(dx, (-1,seq_length,2*FEATURE_LEN)),
        tf.reshape(dx2, (-1,seq_length,2*FEATURE_LEN)),
    ], axis = -1)
    # x.shape == (N, T, 6*FEATURE_LEN) where T <= MAX_FRAME_LEN
    
    # Replace NaN values with zeros
    x = tf.where(tf.math.is_nan(x),tf.constant(0.,x.dtype),x)
    
    return x[0]

table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=list(sign_to_num.keys()),
        values=list(sign_to_num.values()),
    ),
    default_value=tf.constant(-1),
    name="signs"
)

def preprocess_phrase(phrase):
    return table.lookup(phrase)

def preprocess(landmarks, phrase):
    return preprocess_landmarks(landmarks), preprocess_phrase(phrase)

In [23]:
batch_size = 64
train_len = int(0.8 * len(tf_records))

PAD = -100.
NUM_CLASSES = len(sign_to_num)

def get_dataset(tfrecords, batch_size=64, max_len=64, drop_remainder=False, augment=False, shuffle=False, repeat=False, cache=False):
    ds = tf.data.TFRecordDataset(tf_records)
    ds = ds.map(decode_fn, tf.data.AUTOTUNE)
    ds = ds.map(preprocess, tf.data.AUTOTUNE)
    
    if repeat: 
        ds = ds.repeat()
    
    if shuffle:
        ds = ds.shuffle(shuffle)
        options = tf.data.Options()
        options.experimental_deterministic = (False)
        ds = ds.with_options(options)
        
    if batch_size:
        ds = ds.padded_batch(batch_size, padding_values=(PAD, -1), padded_shapes=([MAX_FRAME_LEN, CHANNELS],[]), drop_remainder=drop_remainder)
    
    ds = ds.prefetch(tf.data.AUTOTUNE)
    
    if cache:
        ds = ds.cache()
    
    return ds

train_ds = get_dataset(tf_records[:train_len], batch_size=batch_size)
valid_ds = get_dataset(tf_records[train_len:], batch_size=batch_size)

In [24]:
train_iterator = iter(train_ds)

print("Training Data:")
landmarks, phrase = next(train_iterator)
lm_shape = landmarks.shape[2]

print("Landmarks:")
print(type(landmarks))
print(landmarks.shape)
print(landmarks.numpy())
print("-" * 40)
print("Phrase:")
print(phrase.shape)
print(phrase)
print("-" * 40)

Training Data:
Landmarks:
<class 'tensorflow.python.framework.ops.EagerTensor'>
(64, 128, 372)
[[[-8.46946478e-01  7.90750146e-01 -6.50168478e-01 ... -3.84567678e-03
   -1.35350227e-02 -2.71488726e-03]
  [-8.23016584e-01  7.13746965e-01 -6.19235396e-01 ...  7.18310475e-04
   -1.05634928e-02  2.44282186e-03]
  [-8.01483691e-01  6.73875570e-01 -5.74451506e-01 ...  9.53376293e-05
    1.93512440e-03  1.11070275e-03]
  ...
  [-1.00000000e+02 -1.00000000e+02 -1.00000000e+02 ... -1.00000000e+02
   -1.00000000e+02 -1.00000000e+02]
  [-1.00000000e+02 -1.00000000e+02 -1.00000000e+02 ... -1.00000000e+02
   -1.00000000e+02 -1.00000000e+02]
  [-1.00000000e+02 -1.00000000e+02 -1.00000000e+02 ... -1.00000000e+02
   -1.00000000e+02 -1.00000000e+02]]

 [[-9.77546096e-01  8.21505725e-01 -7.48384178e-01 ...  1.36579275e-02
   -1.77093446e-02  2.47002244e-02]
  [-9.26388502e-01  7.99359977e-01 -6.90003514e-01 ...  3.62835526e-02
   -2.63395160e-02  3.96442413e-02]
  [-8.13614905e-01  7.32428432e-01 -5.767

# Model

In [48]:
# Efficient channel attention
class ECA(tf.keras.layers.Layer):
    def __init__(self, kernel_size=5):
        super().__init__()
        self.supports_masking = True
        self.kernel_size = kernel_size
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False)

    def call(self, inputs, mask=None):
        nn = tf.keras.layers.GlobalAveragePooling1D()(inputs, mask=mask)
        nn = tf.expand_dims(nn, -1)
        nn = self.conv(nn)
        nn = tf.squeeze(nn, -1)
        nn = tf.nn.sigmoid(nn)
        nn = nn[:,None,:]
        return inputs * nn

class CausalDWConv1D(tf.keras.layers.Layer):
    def __init__(self, kernel_size, use_bias=False):
        super().__init__()
        self.supports_masking = True
        
        self.causal_pad = tf.keras.layers.ZeroPadding1D((kernel_size-1, 0))
        self.dw_conv = tf.keras.layers.DepthwiseConv1D(kernel_size, strides=1, padding='valid', use_bias=use_bias)
        
    def call(self, inputs):
        x = self.causal_pad(inputs)
        x = self.dw_conv(x)
        return x

# No dropour noise_shape
def Conv1DBlock(channel_size,
          kernel_size,
          dilation_rate=1,
          drop_rate=0.0,
          expand_ratio=1,
          se_ratio=0.25,
          activation='swish',
          name=None):
    '''
    efficient conv1d block, @hoyso48
    '''
    if name is None:
        name = str(tf.keras.backend.get_uid("mbblock"))
    # Expansion phase
    def apply(inputs):
        channels_in = tf.keras.backend.int_shape(inputs)[-1]
        channels_expand = channels_in * expand_ratio

        skip = inputs
        
        x = tf.keras.layers.Dense(
            channels_expand,
            use_bias=True,
            activation=activation,
            name=name + '_expand_conv')(inputs)

        # Depthwise Convolution
        x = CausalDWConv1D(kernel_size,
            use_bias=False)(x)

        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn')(x)

        x  = ECA()(x)

        x = tf.keras.layers.Dense(
            channel_size,
            use_bias=True,
            name=name + '_project_conv')(x)
        
        if drop_rate > 0:
            x = tf.keras.layers.Dropout(drop_rate, name=name + '_drop')(x)

        if (channels_in == channel_size):
            x = tf.keras.layers.add([x, skip], name=name + '_add')
        return x

    return apply

# same as original (why not the built-in?)
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, dim=256, num_heads=4, dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.scale = self.dim ** -0.5
        self.num_heads = num_heads
        self.qkv = tf.keras.layers.Dense(3 * dim, use_bias=False)
        self.drop1 = tf.keras.layers.Dropout(dropout)
        self.proj = tf.keras.layers.Dense(dim, use_bias=False)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        qkv = self.qkv(inputs)
        qkv = tf.keras.layers.Permute((2, 1, 3))(tf.keras.layers.Reshape((-1, self.num_heads, self.dim * 3 // self.num_heads))(qkv))
        q, k, v = tf.split(qkv, [self.dim // self.num_heads] * 3, axis=-1)

        attn = tf.matmul(q, k, transpose_b=True) * self.scale

        if mask is not None:
            mask = mask[:, None, None, :]

        attn = tf.keras.layers.Softmax(axis=-1)(attn, mask=mask)
        attn = self.drop1(attn)

        x = attn @ v
        x = tf.keras.layers.Reshape((-1, self.dim))(tf.keras.layers.Permute((2, 1, 3))(x))
        x = self.proj(x)
        return x

def TransformerBlock(dim=256, num_heads=4, expand=4, attn_dropout=0.2, drop_rate=0.2, activation='swish'):
    def apply(inputs):
        x = inputs
        x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
        x = MultiHeadSelfAttention(dim=dim,num_heads=num_heads,dropout=attn_dropout)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([inputs, x])
        attn_out = x

        x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
        x = tf.keras.layers.Dense(dim*expand, use_bias=False, activation=activation)(x)
        x = tf.keras.layers.Dense(dim, use_bias=False)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([attn_out, x])
        return x
    return apply

# small dropout
def get_model(dim):
    ksize = 17

    inp = tf.keras.Input((MAX_FRAME_LEN,CHANNELS))
    x = tf.keras.layers.Masking(mask_value=PAD,input_shape=(MAX_FRAME_LEN,CHANNELS))(inp)
    x = tf.keras.layers.Dense(dim, use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
    
    x = Conv1DBlock(dim, ksize, drop_rate=0.2)(x)
    x = Conv1DBlock(dim, ksize, drop_rate=0.2)(x)
    x = Conv1DBlock(dim, ksize, drop_rate=0.2)(x)
    x = Conv1DBlock(dim, ksize, drop_rate=0.2)(x)
    x = TransformerBlock(dim, expand=2)(x)

    x = tf.keras.layers.Dense(dim*2, activation=None)(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(NUM_CLASSES)(x)
    return tf.keras.Model(inp, x)

In [49]:
lm_seq, sing_idx = next(iter(train_ds))

In [50]:
model = get_model(192)
y = model(lm_seq)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 128, 372)]           0         []                            
                                                                                                  
 masking_7 (Masking)         (None, 128, 372)             0         ['input_8[0][0]']             
                                                                                                  
 dense_67 (Dense)            (None, 128, 192)             71424     ['masking_7[0][0]']           
                                                                                                  
 batch_normalization_48 (Ba  (None, 128, 192)             768       ['dense_67[0][0]']            
 tchNormalization)                                                                          

In [51]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"])

In [52]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='first_run_till_its_dead.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    verbose=2
)

In [53]:
%%time
model.fit(train_ds,
          validation_data=valid_ds,
          epochs=50,
          #callbacks=[checkpoint_callback],
          verbose = 2)

Epoch 1/50


KeyboardInterrupt: 